In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

trust = pd.read_csv("/content/drive/MyDrive/Implicit/Ciao/implicitTrust.csv")
trust = trust[["from","to","value"]]

In [ ]:
train = pd.read_csv("/content/drive/MyDrive/Implicit/Ciao/trainImplicit.csv")
train = train[["userId","productId","rating"]]
train

userId  productId  rating
0          692      38404       1
1          850       6956       1
2          245      17754       1
3         1084      18155       1
4         1401      40830       1
...        ...        ...     ...
181908    2041      63223       1
181909     584      36373       1
181910      45       4765       1
181911    6270      47274       1
181912    2658      68970       1

[181913 rows x 3 columns]

In [ ]:
x = train.groupby("userId")
users = list(x.groups.keys())

In [ ]:
import numpy as np
from scipy.sparse import csc_matrix
mat_data = csc_matrix((7373,7373), dtype=np.int8)
mat_data = mat_data.tolil()
print(mat_data.shape)
for uid,pid,_ in trust.values:
  mat_data[uid,pid] = 1
#mat_data = mat_data.tocoo()

(7373, 7373)


In [ ]:
import numpy as np
user_train_popularity = mat_data[:, :].toarray().sum(axis=1)
np.savetxt("/content/drive/MyDrive/Implicit/Ciao/user_popularity.txt",user_train_popularity)

In [ ]:
user_neighbor_items = {}
user_nei_pop = {}
y = trust.groupby("from")
from_users = list(y.groups.keys())
for i in range(len(from_users)):
  neighbors = y.get_group(from_users[i]).values[:,1]
  items = []
  nei_pop = []
  for j in range(len(neighbors)):
    if(neighbors[j]==from_users[i]):
      continue
    try:
      temp = [user_train_popularity[neighbors[j]]]*len(x.get_group(neighbors[j]).values[:,1])
      nei_pop.extend(temp)
      items.extend(x.get_group(neighbors[j]).values[:,1])
    except:
      continue
  items = list(items)
  user_neighbor_items[from_users[i]]=items
  user_nei_pop[from_users[i]] = nei_pop

In [ ]:
len(user_nei_pop[0])

6122

In [ ]:
len(user_neighbor_items[0])

6122

In [ ]:
len(list(user_nei_pop.keys()))

7373

In [ ]:
len(user_nei_pop[0])-len(user_neighbor_items[0])

0

In [ ]:
len(list(user_neighbor_items.keys()))

7373

In [ ]:
import pickle
with open('/content/drive/MyDrive/Implicit/Ciao/user_neighbor_items_test.pickle', 'wb') as handle:
    pickle.dump(user_neighbor_items, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
import pickle
with open('/content/drive/MyDrive/Implicit/Ciao/user_neighbor_popularity_test.pickle', 'wb') as handle:
    pickle.dump(user_nei_pop, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
import numpy as np

train_popularity = np.loadtxt("/content/drive/MyDrive/Implicit/Epinions/Data/train_popularity.txt")


139738

In [ ]:
import numpy as np
from scipy.sparse import csc_matrix
mat_data = csc_matrix((37952,119015), dtype=np.int8)
mat_data = mat_data.tolil()
print(mat_data.shape)
for uid,pid,_ in train.values:
  mat_data[uid,pid] = 1
#mat_data = mat_data.tocoo()

(37952, 119015)


In [ ]:
np.savetxt("/content/drive/MyDrive/Implicit/Epinions/Data/train_popularity_1.txt",np.array(mat_data.sum(axis=0))[0])